In [82]:
import os
import json
import boto3
import pandas as pd
import numpy as np
from boto3.dynamodb.types import TypeSerializer, TypeDeserializer
from boto3.dynamodb.conditions import Key
from decimal import Decimal

In [83]:
from dotenv import load_dotenv

In [84]:
load_dotenv()

True

In [85]:

ts= TypeSerializer()
td = TypeDeserializer()

class DecimalEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Decimal):
      return str(obj)
    return json.JSONEncoder.default(self, obj)

In [141]:
def query_sql(beer_id, dynamodb=None):
    if not dynamodb:
        dyna_clnt=boto3.client('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))
    stmt = "SELECT BREW_ID,PI_TMSTP,STAGE,location,temperature,thermo_loc,thermo_temp FROM BREW_MON WHERE BREW_ID='" + beer_id + "'  ORDER BY PI_TMSTP DESC"
   # print(lambda_handler(event=brew1))
    response = dyna_clnt.execute_statement(Statement=stmt)
    print("HI")
    return response    
 

In [142]:
lines = query_sql(beer_id='SCWG_IPA')

HI


In [88]:
print(lines)

{'Items': [{'BREW_ID': {'S': 'SCWG_IPA'}, 'PI_TMSTP': {'S': '2021-09-17 16:30:12'}, 'STAGE': {'S': 'FERMENT'}, 'location': {'S': 'STUDY'}, 'temperature': {'S': '21.0'}, 'thermo_loc': {'S': 'IN_BEER'}, 'thermo_temp': {'N': '17.65'}}, {'BREW_ID': {'S': 'SCWG_IPA'}, 'PI_TMSTP': {'S': '2021-09-17 16:20:11'}, 'STAGE': {'S': 'FERMENT'}, 'location': {'S': 'STUDY'}, 'temperature': {'S': '22.0'}, 'thermo_loc': {'S': 'IN_BEER'}, 'thermo_temp': {'N': '17.73'}}, {'BREW_ID': {'S': 'SCWG_IPA'}, 'PI_TMSTP': {'S': '2021-09-17 16:10:11'}, 'STAGE': {'S': 'FERMENT'}, 'location': {'S': 'STUDY'}, 'temperature': {'S': '21.0'}, 'thermo_loc': {'S': 'IN_BEER'}, 'thermo_temp': {'N': '17.65'}}, {'BREW_ID': {'S': 'SCWG_IPA'}, 'PI_TMSTP': {'S': '2021-09-17 16:00:11'}, 'STAGE': {'S': 'FERMENT'}, 'location': {'S': 'STUDY'}, 'temperature': {'S': '21.0'}, 'thermo_loc': {'S': 'IN_BEER'}, 'thermo_temp': {'N': '17.82'}}, {'BREW_ID': {'S': 'SCWG_IPA'}, 'PI_TMSTP': {'S': '2021-09-17 15:50:11'}, 'STAGE': {'S': 'FERMENT'}, '

In [143]:
 df_raw = pd.json_normalize(lines, record_path =['Items'])

In [144]:
print(df_raw)

     BREW_ID.S           PI_TMSTP.S  STAGE.S location.S temperature.S  \
0     SCWG_IPA  2021-09-17 16:30:12  FERMENT      STUDY          21.0   
1     SCWG_IPA  2021-09-17 16:20:11  FERMENT      STUDY          22.0   
2     SCWG_IPA  2021-09-17 16:10:11  FERMENT      STUDY          21.0   
3     SCWG_IPA  2021-09-17 16:00:11  FERMENT      STUDY          21.0   
4     SCWG_IPA  2021-09-17 15:50:11  FERMENT      STUDY          21.0   
...        ...                  ...      ...        ...           ...   
1616  SCWG_IPA  2021-09-05 18:30:11  FERMENT      study            16   
1617  SCWG_IPA  2021-09-05 18:20:11  FERMENT      study            16   
1618  SCWG_IPA  2021-09-05 18:10:11  FERMENT      study            16   
1619  SCWG_IPA  2021-09-05 18:00:11  FERMENT      study            16   
1620  SCWG_IPA  2021-09-05 17:50:11  FERMENT      study            16   

     thermo_loc.S thermo_temp.N  
0         IN_BEER         17.65  
1         IN_BEER         17.73  
2         IN_BEER    

In [145]:
    df_raw['pi_time']=pd.to_datetime(df_raw['PI_TMSTP.S'])
    df_raw['temperature_n']=pd.to_numeric(df_raw['temperature.S'], downcast="float")
    df_raw['thermo_temp_n']=pd.to_numeric(df_raw['thermo_temp.N'], downcast="float")    
    df_raw.set_index('pi_time',inplace=True)

In [146]:
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n > 50].index)
    df_raw = df_raw.drop(df_raw[df_raw.thermo_temp_n < 0].index)
    df_raw = df_raw.drop(columns=['BREW_ID.S','PI_TMSTP.S','STAGE.S','location.S','thermo_loc.S','temperature.S','thermo_temp.N'])
    df_raw['temperature_n'] = df_raw['temperature_n'].round(decimals=2)
    df_raw['thermo_temp_n'] = df_raw['thermo_temp_n'].round(decimals=2)

In [147]:
print(df_raw)

                     temperature_n  thermo_temp_n
pi_time                                          
2021-09-17 16:30:12           21.0          17.65
2021-09-17 16:20:11           22.0          17.73
2021-09-17 16:10:11           21.0          17.65
2021-09-17 16:00:11           21.0          17.82
2021-09-17 15:50:11           21.0          17.65
...                            ...            ...
2021-09-05 18:30:11           16.0            NaN
2021-09-05 18:20:11           16.0            NaN
2021-09-05 18:10:11           16.0            NaN
2021-09-05 18:00:11           16.0            NaN
2021-09-05 17:50:11           16.0            NaN

[1594 rows x 2 columns]


In [153]:
    df_4h = df_raw.groupby([pd.Grouper(freq='2H', level='pi_time')]).mean(numeric_only=True)
    df_4h = df_4h[df_4h['temperature_n'].notna()]
    df_4h = df_4h[df_4h['thermo_temp_n'].notna()]
    df_4h[['temperature_n','thermo_temp_n']] = df_4h[['temperature_n','thermo_temp_n']].round(2)
    #df_4h['thermo_temp_n'] = df_4h['thermo_temp_n'].round(decimals=2)

In [105]:
   df_4h['pi_time'] = df_4h['pi_time'].astype(str)

In [154]:
print(df_4h)

                     temperature_n  thermo_temp_n
pi_time                                          
2021-09-08 00:00:00      13.250000      17.820000
2021-09-09 22:00:00      20.110001      19.070000
2021-09-10 00:00:00      18.500000      19.100000
2021-09-10 02:00:00      18.000000      19.049999
2021-09-10 04:00:00      17.250000      19.040001
...                            ...            ...
2021-09-17 08:00:00      16.920000      17.520000
2021-09-17 10:00:00      19.250000      17.480000
2021-09-17 12:00:00      20.330000      17.480000
2021-09-17 14:00:00      21.080000      17.629999
2021-09-17 16:00:00      21.250000      17.709999

[94 rows x 2 columns]


In [155]:
    df_4h=df_4h.reset_index()
  

In [64]:
  df_4h_json = df_4h.to_dict(orient='records',date_format='iso')

TypeError: DataFrame.to_dict() got an unexpected keyword argument 'date_format'

In [ ]:
    json_4h = {'Items': df_4h_json}

In [63]:
print(df_4h_json)


[{"index":0,"pi_time":"2021-09-08T00:00:00.000","temperature_n":13.25,"thermo_temp_n":17.8199996948},{"index":1,"pi_time":"2021-09-09T22:00:00.000","temperature_n":20.1111106873,"thermo_temp_n":19.0737495422},{"index":2,"pi_time":"2021-09-10T00:00:00.000","temperature_n":18.5,"thermo_temp_n":19.1033325195},{"index":3,"pi_time":"2021-09-10T02:00:00.000","temperature_n":18.0,"thermo_temp_n":19.0524997711},{"index":4,"pi_time":"2021-09-10T04:00:00.000","temperature_n":17.25,"thermo_temp_n":19.0399990082},{"index":5,"pi_time":"2021-09-10T06:00:00.000","temperature_n":17.1666660309,"thermo_temp_n":18.9750003815},{"index":6,"pi_time":"2021-09-10T08:00:00.000","temperature_n":18.75,"thermo_temp_n":19.0249996185},{"index":7,"pi_time":"2021-09-10T10:00:00.000","temperature_n":21.3333339691,"thermo_temp_n":18.9958324432},{"index":8,"pi_time":"2021-09-10T12:00:00.000","temperature_n":21.75,"thermo_temp_n":19.0816669464},{"index":9,"pi_time":"2021-09-10T14:00:00.000","temperature_n":22.5833339691,

In [136]:

new_items = df_4h.to_dict(orient='records')
  


In [137]:
print(new_items)

[{'temperature_n': 13.25, 'thermo_temp_n': 17.81999969482422}, {'temperature_n': 20.110000610351562, 'thermo_temp_n': 19.06999969482422}, {'temperature_n': 18.5, 'thermo_temp_n': 19.100000381469727}, {'temperature_n': 18.0, 'thermo_temp_n': 19.049999237060547}, {'temperature_n': 17.25, 'thermo_temp_n': 19.040000915527344}, {'temperature_n': 17.170000076293945, 'thermo_temp_n': 18.979999542236328}, {'temperature_n': 18.75, 'thermo_temp_n': 19.020000457763672}, {'temperature_n': 21.329999923706055, 'thermo_temp_n': 19.0}, {'temperature_n': 21.75, 'thermo_temp_n': 19.079999923706055}, {'temperature_n': 22.579999923706055, 'thermo_temp_n': 19.09000015258789}, {'temperature_n': 22.670000076293945, 'thermo_temp_n': 19.06999969482422}, {'temperature_n': 19.329999923706055, 'thermo_temp_n': 18.709999084472656}, {'temperature_n': 18.0, 'thermo_temp_n': 17.309999465942383}, {'temperature_n': 17.920000076293945, 'thermo_temp_n': 17.979999542236328}, {'temperature_n': 17.0, 'thermo_temp_n': 18.129

In [26]:
from datetime import datetime

In [ ]:
for item in new_items:
   # item['TEMP_DATE'] = datetime.strptime(item['pi_time'], '%Y-%m-%dT%H:%M:%SZ').isoformat()
    response = table.update_item(
        Key={
            'itemName': 'SCWP_IPA'  # Assuming 'itemName' is the partition key in your table
        },
        UpdateExpression="SET #data = list_append(if_not_exists(#data, :empty_list), :vals)",
        ExpressionAttributeNames={
            '#data': 'TEMP_DATE'
        },
        ExpressionAttributeValues={
            ':vals': [item],
            ':empty_list': []
        },
        ReturnValues="UPDATED_NEW"
    )
    print("Item updated:", response)

In [ ]:
print(df_4h_json)

In [ ]:
        dyna_clnt=boto3.client('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))

In [ ]:

# Initialize DynamoDB client
dynamodb = boto3.resource('dynamodb',
                               region_name=os.getenv('AWS_REGION'),         
                               aws_access_key_id=os.getenv('AWS_KEY'),          
                               aws_secret_access_key=os.getenv('AWS_SECRET_KEY'))



In [ ]:
table = dynamodb.Table('BREW_LOG')

In [ ]:
print(json)

In [37]:
data = {
    'temperature': [60, 62, 65, 63, 61],
    'timestamp': [
        '2023-12-18T00:00:00Z',
        '2023-12-18T02:00:00Z',
        '2023-12-18T04:00:00Z',
        '2023-12-18T06:00:00Z',
        '2023-12-18T08:00:00Z'
    ]
}

In [50]:
data2 = {
    'temperature': [60, 62, 65, 63, 61],
    'timestamp': [
        '2023-12-18 00:00:00',
        '2023-12-18 02:00:00',
        '2023-12-18 04:00:00',
        '2023-12-18 06:00:00',
        '2023-12-18 08:00:00'
    ]
}

In [128]:
test_date ={
    'pi_time': ['2021-09-08 00:00:00','2021-09-09 22:00:00'], 
'temperature_n': [1325,20],
'thermo_temp_n': [17, 19]
    
}

In [129]:
print(test_date)

{'pi_time': ['2021-09-08 00:00:00', '2021-09-09 22:00:00'], 'temperature_n': [1325, 20], 'thermo_temp_n': [17, 19]}


In [130]:
df_tss = pd.DataFrame(test_date)
print(df_tss)
items_test3 = df_tss.to_dict(orient='records')

               pi_time  temperature_n  thermo_temp_n
0  2021-09-08 00:00:00           1325             17
1  2021-09-09 22:00:00             20             19


In [52]:

df = pd.DataFrame(data2)

# Convert DataFrame to a list of dictionaries
items_test = df.to_dict(orient='records')

In [123]:
print(items_test3)
print('0----------------------------------------------')
print(items_test)

[{'pi_time': '2021-09-08 00:00:00', 'temperature_n': 13.25, 'thermo_temp_n': 17.81999969482422}, {'pi_time': '2021-09-09 22:00:00', 'temperature_n': 20.11111068725586, 'thermo_temp_n': 19.073749542236328}]
0----------------------------------------------
[{'temperature': 60, 'timestamp': '2023-12-18 00:00:00'}, {'temperature': 62, 'timestamp': '2023-12-18 02:00:00'}, {'temperature': 65, 'timestamp': '2023-12-18 04:00:00'}, {'temperature': 63, 'timestamp': '2023-12-18 06:00:00'}, {'temperature': 61, 'timestamp': '2023-12-18 08:00:00'}]


In [55]:
print(df_4h)

               pi_time  temperature_n  thermo_temp_n
0  2021-09-08 00:00:00      13.250000      17.820000
1  2021-09-09 22:00:00      20.111111      19.073750
2  2021-09-10 00:00:00      18.500000      19.103333
3  2021-09-10 02:00:00      18.000000      19.052500
4  2021-09-10 04:00:00      17.250000      19.039999
..                 ...            ...            ...
89 2021-09-17 08:00:00      16.916666      17.522499
90 2021-09-17 10:00:00      19.250000      17.477499
91 2021-09-17 12:00:00      20.333334      17.484999
92 2021-09-17 14:00:00      21.083334      17.633333
93 2021-09-17 16:00:00      21.250000      17.712500

[94 rows x 3 columns]


In [151]:
items_test2 = df_4h.to_dict(orient='records')
print(items_test2)

[{'pi_time': Timestamp('2021-09-08 00:00:00'), 'temperature_n': 13.25, 'thermo_temp_n': 17.81999969482422}, {'pi_time': Timestamp('2021-09-09 22:00:00'), 'temperature_n': 20.110000610351562, 'thermo_temp_n': 19.06999969482422}, {'pi_time': Timestamp('2021-09-10 00:00:00'), 'temperature_n': 18.5, 'thermo_temp_n': 19.100000381469727}, {'pi_time': Timestamp('2021-09-10 02:00:00'), 'temperature_n': 18.0, 'thermo_temp_n': 19.049999237060547}, {'pi_time': Timestamp('2021-09-10 04:00:00'), 'temperature_n': 17.25, 'thermo_temp_n': 19.040000915527344}, {'pi_time': Timestamp('2021-09-10 06:00:00'), 'temperature_n': 17.170000076293945, 'thermo_temp_n': 18.979999542236328}, {'pi_time': Timestamp('2021-09-10 08:00:00'), 'temperature_n': 18.75, 'thermo_temp_n': 19.020000457763672}, {'pi_time': Timestamp('2021-09-10 10:00:00'), 'temperature_n': 21.329999923706055, 'thermo_temp_n': 19.0}, {'pi_time': Timestamp('2021-09-10 12:00:00'), 'temperature_n': 21.75, 'thermo_temp_n': 19.079999923706055}, {'pi_t

In [152]:
table.update_item(
   Key={"BEER_ID":"SCWG_IPA", "BEER_DATE": "2021-09-05"},
                UpdateExpression="set TEMP_DATA3=:r",
                ExpressionAttributeValues={":r": items_test2},
                ReturnValues="UPDATED_NEW"
)

TypeError: Unsupported type "<class 'pandas._libs.tslibs.timestamps.Timestamp'>" for value "2021-09-08 00:00:00"

In [115]:
print(items_test)
print(items_test2)

[{'temperature': 60, 'timestamp': '2023-12-18 00:00:00'}, {'temperature': 62, 'timestamp': '2023-12-18 02:00:00'}, {'temperature': 65, 'timestamp': '2023-12-18 04:00:00'}, {'temperature': 63, 'timestamp': '2023-12-18 06:00:00'}, {'temperature': 61, 'timestamp': '2023-12-18 08:00:00'}]
[{'pi_time': '2021-09-08 00:00:00', 'temperature_n': 13.25, 'thermo_temp_n': 17.81999969482422}, {'pi_time': '2021-09-09 22:00:00', 'temperature_n': 20.11111068725586, 'thermo_temp_n': 19.073749542236328}, {'pi_time': '2021-09-10 00:00:00', 'temperature_n': 18.5, 'thermo_temp_n': 19.10333251953125}, {'pi_time': '2021-09-10 02:00:00', 'temperature_n': 18.0, 'thermo_temp_n': 19.052499771118164}, {'pi_time': '2021-09-10 04:00:00', 'temperature_n': 17.25, 'thermo_temp_n': 19.03999900817871}, {'pi_time': '2021-09-10 06:00:00', 'temperature_n': 17.16666603088379, 'thermo_temp_n': 18.975000381469727}, {'pi_time': '2021-09-10 08:00:00', 'temperature_n': 18.75, 'thermo_temp_n': 19.024999618530273}, {'pi_time': '20

In [33]:
nested_preferences = {
    "theme": "dark",
    "notifications": {
        "email": True,
        "sms": False
    },
    "settings": {
        "language": "en",
        "timezone": "GMT+0"
    }
}

# Example user item with nested preferences
user_item = {
'BEER_ID':"TESTSCWG_IPA", 'BEER_DATE': "2021-09-05",
    'age': 30,
    'preferences': nested_preferences
}

# Get reference to the DynamoDB table
table = dynamodb.Table("BREW_LOG")

# Put the item into the table
table.put_item(Item=user_item)

{'ResponseMetadata': {'RequestId': '6T2B820UEE29KPHCUA7IS9L7VBVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 19 Dec 2023 14:42:35 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6T2B820UEE29KPHCUA7IS9L7VBVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [35]:
table.update_item(
    Key={'BEER_ID':"TESTSCWG_IPA", 'BEER_DATE': "2021-09-05"},
    AttributeUpdates={
        'status': 'complete'
    }
)

ParamValidationError: Parameter validation failed:
Invalid type for parameter AttributeUpdates.status, value: complete, type: <class 'str'>, valid types: <class 'dict'>